> **Downloading the COCO Dataset Annotations File**

In [4]:
import requests
import zipfile
import os

# URL for the annotation file
url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
save_path = "annotations_trainval2017.zip"

# Download the zip file
response = requests.get(url, stream=True)
with open(save_path, "wb") as f:
    f.write(response.content)

# Extract the zip file
with zipfile.ZipFile(save_path, 'r') as zip_ref:
    zip_ref.extractall("annotations")

# Path to the JSON file
annotation_path = "annotations/instances_val2017.json"
print(f"Annotation file is available at: {annotation_path}")

Annotation file is available at: annotations/instances_val2017.json


> **Downloading the COCO Dataset Images using the Annotations File into a folder called 'colored_images'**

In [ ]:
from pycocotools.coco import COCO
import requests
import os

# Initialize COCO dataset
coco = COCO("annotations/instances_val2017.json")

# Get all image IDs
img_ids = coco.getImgIds()  # No filters
print(f"Number of Images: {len(img_ids)}")

if len(img_ids) > 0:
    # Directory to save images
    save_path = "colored_images"
    os.makedirs(save_path, exist_ok=True)

    # Download images
    for img_id in img_ids[:10000]:  # Limit to 10 for demonstration
        try:
            img_info = coco.loadImgs(img_id)[0]  # Get image metadata
            img_url = img_info['coco_url']      # Get the image URL
            print(f"Downloading {img_info['file_name']} from {img_url}")

            # Download the image
            response = requests.get(img_url, stream=True)
            if response.status_code == 200:
                # Save the image to the directory
                with open(os.path.join(save_path, img_info['file_name']), 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
            else:
                print(f"Failed to download {img_info['file_name']}. HTTP Status Code: {response.status_code}")
        except Exception as e:
            print(f"Error downloading image {img_id}: {e}")
else:
    print("No images found in the dataset.")

loading annotations into memory...
Done (t=0.76s)
creating index...
index created!
Number of Images: 5000


> **Before this step, download the zipped file from the following URL: https://drive.google.com/file/d/1iOt3PCIrLGXMtbNN-p-PdV6OFz-KyqLp/view?usp=drive_link**
>
> **Extract the downloaded zipped file into a folder 'selected_images'**
>
> **Run the following code block, to augment the selected images into a folder called 'filtered_iamges'**

In [1]:
import os
from PIL import Image, ImageEnhance
import random

# Define augmentation functions
def augment_image(image):
    # Randomly apply transformations
    if random.choice([True, False]):
        image = image.rotate(random.uniform(-30, 30))  # Rotate
    if random.choice([True, False]):
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(random.uniform(0.7, 1.3))  # Adjust brightness
    if random.choice([True, False]):
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(random.uniform(0.7, 1.3))  # Adjust contrast
    if random.choice([True, False]):
        image = image.transpose(random.choice([Image.FLIP_LEFT_RIGHT, Image.FLIP_TOP_BOTTOM]))  # Flip
    return image

# Augment images in folder
def augment_images_in_folder(input_folder, output_folder, augment_count=3):
    os.makedirs(output_folder, exist_ok=True)
    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith(('png', 'jpg', 'jpeg')):
            file_path = os.path.join(input_folder, file_name)
            image = Image.open(file_path)
            for i in range(augment_count):
                augmented_image = augment_image(image.copy())
                output_path = os.path.join(output_folder, f"{os.path.splitext(file_name)[0]}_aug{i}.png")
                augmented_image.save(output_path)

# Example usage
input_folder = "selected_images"
output_folder = "filtered_images"
augment_images_in_folder(input_folder, output_folder, augment_count=5)

>**Before this step, copy all the images from 'filtered_images' folder into 'colored_images' folder.**
>
>**Run the following code to convert colored images to grayscale images into 'graycale_images' folder.**

In [3]:
from PIL import Image
import os

# Paths for input and target directories
input_dir = "grayscale_images"
target_dir = "colored_images"
os.makedirs(input_dir, exist_ok=True)

# Process images
for img_file in os.listdir(target_dir):
    img_path = os.path.join(target_dir, img_file)
    img = Image.open(img_path)

    # Handle grayscale images
    if img.mode != "RGB":
        print(f"Converting grayscale to RGB: {img_file}")
        img = img.convert("RGB")  # Convert grayscale to RGB

    # Convert to grayscale for input
    gray_img = img.convert("L")
    gray_img.save(os.path.join(input_dir, img_file))

Converting grayscale to RGB: 000000205289.jpg
